<a href="https://colab.research.google.com/github/VishalPrem1994/AiGenPlayground/blob/main/Udemy_Course_1_Running_On_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
#@markdown Login to HuggingFace 

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
#@markdown Specify the weights directory to use 
WEIGHTS_DIR = "stable_diffusion_weights/vishal/2720" #@param {type:"string"}
WEIGHTS_DIR = "/content/drive/MyDrive/" + WEIGHTS_DIR
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")
from google.colab import drive
drive.mount('/content/drive')

[*] WEIGHTS_DIR=/content/drive/MyDrive/stable_diffusion_weights/vishal/2720
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = None

#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

#@title Run for generating images.

prompt = "photo of vishal, cyberpunk, (((fit))), ((perfect symmetrical face)), ultra realistic, concept art, ((intricate)), ((highly detailed)), depth of field, ((professionally color graded)), soft ambient lighting, dusk, 8k, centered, art by artgerm and greg rutkowski and alphonse mucha " #@param {type:"string"}
negative_prompt = "far away, blurry face, wrong eyes, extra hands, extra legs, wierd eyes, disfigured face, (((backlight))), (((dark face))), white fur, (gold fabric), (gold dress), ((cropped head)), ((out of frame)), ((long neck)), deformed, cripple, ugly, additional arms, additional legs, additional head, two heads, multiple people, group of people" #@param {type:"string"}
num_samples = 1 #@param {type:"number"}
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 20 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)